# NE 250 – Homework 6
## Problem 4
###### 12/1/2017

In [1]:
import numpy as np
import random

We are considering an infinite, steady-state, monoenergetic, two-region Monte Carlo problem with the following characteristics: (note thate we have renamed region 1 and region 2 as region 0 and region 1 respectively; this allows for simpler calculations)
* 1-D problem geometry
* Region 0 has $\Sigma_s = 0.5\text{ cm}^{-1}$ and $\Sigma_t = 1.0\text{ cm}^{-1}$ (in both regions the only interactions are
scattering or absorption).
* Region 1 has $\Sigma_s = 0.75\text{ cm}^{-1}$ and $\Sigma_t =0.9\text{ cm}^{-1}$.
* Region 0 has $w_{nom} = 1$ and Region 1 has $w_{nom} = 2$. $w_{max}$ and $w_{min}$ can be found as $(w_{nom} \times 2.5)$ or $(w_{nom}\,/\,2.5)$, respectively.
* All particles are born in Region 0 with weight 1 at a location that is 1 cm to the left of the interface between Regions 0 and 1. The source is isotropic.
* Isotropic scattering.

#### Problem Statement 

_Write the algorithm for a Monte Carlo code to solve this specific problem. Include the PDFs required for sampling as well as algorithms for conducting sampling. Use a collision estimator to tally the flux. Include implicit capture, rouletting, and splitting._

### Underlying parameters

Using the above information, we can create a set of dictionaries to describe our physical situation. Each fundamental parameter gets a dictionary, and each dictionary has entries corresponding to each region.

In [2]:
# Macroscopic Total Cross Sections
Sigma_t = {0:1.0,1:0.9}

# Macroscopic Scattering Cross Sections
Sigma_s = {0:0.5,1:0.75}

# Nominal Weights 
w_nom = {0:1,1:2}

# Max/Min Weights
w_ext = {region:(w_nom[region]*2.5,w_nom[region]/2.5) for region in w_nom.keys()}

In [3]:
w_ext

{0: (2.5, 0.4), 1: (5.0, 0.8)}

### Tracking the particle

We can track a particle using a particle class, which can model the behavior of each particle as it proceeds through it's lifetime. This particle class will contain methods for each action that the particle will undergo
* birth (the class' `_init__` method)
* transport
* boundary encounter
* collision
* scoring

## Survival Biasing

In [4]:
class particle:
    """
    A class to model a single Monte Carlo particle over it's lifetime
    """
    def __init__(self,verbose=False):
        """The particle is born. Assign a position [cm], angle (cos θ), energy [MeV], and weight."""
        self.x = -1
        self.mu = 2*random.random()-1
        self.E = 1
        self.w = 1
        self.region = 0
        self.score = np.zeros(2)
        self.verbose = verbose
        if verbose:
            print('The particle was born at x = -1 cm (region 0), with weight 1.')
            
    def transport(self,sample=True):
        """Transport the particle through the problem geometry"""
        # Sample to find the number of mean free paths that traveled by the particle material; adjust for 1 dimension
        if sample:
            xi = random.random()
            self.mfp_x = -np.log(xi)*self.mu
            if self.verbose: print('This particle will travel {} MFPs in the x-direction.'.format(round(self.mfp_x,3)))
        # Determine the number of mean free paths to a boundary in the current material
        boundary_mfp = -self.x*Sigma_t[self.region]
        if self.verbose: print('The distance to the boundary is {} MFPs.'.format(round(boundary_mfp,3)))
        # If the particle reaches a boundary before the collision, stop and reevaluate
        if boundary_mfp == 0:  # (the particle was at the boundary, so must collide)
            if self.verbose: print('(the particle is at the boundary)')
            self.collision()
        elif self.mfp_x > boundary_mfp:
            self.boundary(boundary_mfp)
        else:
            self.collision()
    
    def boundary(self,boundary_mfp):
        """The particle reached a boundary: reevaluate the particle's tracking in the new material"""
        if self.verbose: print('The particle travels to the boundary.')
        self.x = 0
        self.region = 1-self.region
        self.mfp_x -= boundary_mfp
        self.update_weight(collision=False)
        self.transport(sample=False)
        
    def collision(self):
        """The particle collided: use survival biasing to continue following the particle"""
        self.x += self.mfp_x
        if self.verbose: print('The particle collides at x = {}'.format(round(self.x,3)))
        self.score_particle()
        self.update_weight(collision=True)
        if self.w == 0: 
            if self.verbose: print('\t\t\t\t The particle is killed (w=0).')
            return
        self.mu = 2*random.random()-1
        if self.verbose: print('The particle scatters, and is now traveling with mu = {}'.format(round(self.mu,3)))
        self.transport(sample=True)  
    
    def update_weight(self,collision=False):
        """Update the particle's weight using survival biasing, splitting, and russian roulette"""
        if collision: # adjust weight with survival biasing
            # New weight:     w_{i+1} = w(1 - Σ_a / Σ_t)
            self.w *= (Sigma_t[self.region]-Sigma_s[self.region])/Sigma_t[self.region]
            if self.verbose: print('The particle is survival biased, now with weight {}'.format(round(self.w,3)))
        # Splitting
        if self.w > w_ext[self.region][0]:
            SR = self.w/w_nom[self.region]
            xi = random.random()
            if xi >= SR - int(SR):
                b = 0
            else:
                b = 1
            n_new_particles = int(SR) + b
            if self.verbose: print('Splitting the particle into {} new particles.'.format(n_new_particles))
            global particles
            for i in range(int(SR)):
                particles.append(particle())
                particles[-1].x = self.x
                particles[-1.].w = self.w/n_new_particles
        # Russian Roulette
        if self.w < w_ext[self.region][1]:
            if self.verbose: print('Rouletting the particles...')
            xi = random.random()
            RR = self.w/w_nom[self.region]
            if xi >= RR :
                self.w = 0
                if self.verbose: print('\t\t\t ... uh-oh...')
            else:
                self.w = w_nom[self.region]
                if self.verbose: print('Phew. Particle survived, and now it has weight {}'.format(round(self.w,3)))
        
    def score_particle(self):
        self.score[self.region] += self.w
        if self.verbose: 
            print('Score! Particle with weight {} added to the tally.'.format(round(self.w,3)))
            print('\tCurrent score: \t Region 0: {}   Region 1: {}'.format(self.score[0],self.score[1]))

        

In [5]:
## Survival biasing, rouletting, and splitting MC run
N = 20000
tally = np.zeros(2)
particles = [particle() for n in range(N)]
for p in particles:
    p.transport()
    tally += p.score
norm_collisions = tally/N

In [6]:
print('Average collisions per particle, region 0: {} \t Average collisions per particle,, region 1: {}'.format(norm_collisions[0],norm_collisions[1]))
print('Ratio: ',norm_collisions[0]/norm_collisions[1])

Average collisions per particle, region 0: 1.728825 	 Average collisions per particle,, region 1: 0.1598
Ratio:  10.8186795995


Interestingly, it appears that in our specific problem splitting is never used. This fact could be deduced, however, by noting that splitting only occurs when $w_i > w_{max}$. For both regions, $w_{max} \geq 2.5$. Upon birth in either region, a particle's weight will never be more than 2. Collisions will only trigger a reduction in weight, and rouletting produces particles also with a maximum weight of 2 (roulette products have weight $w_{nom}$).

## The story of a Monte Carlo Particle

Let's follow just 1 particle.

In [7]:
random.seed(4)
N = 1
tally = np.zeros(2)
p = particle(verbose=True)
p.transport()
tally += p.score

The particle was born at x = -1 cm (region 0), with weight 1.
This particle will travel -1.199 MFPs in the x-direction.
The distance to the boundary is 1.0 MFPs.
The particle collides at x = -2.199
Score! Particle with weight 1 added to the tally.
	Current score: 	 Region 0: 1.0   Region 1: 0.0
The particle is survival biased, now with weight 0.5
The particle scatters, and is now traveling with mu = -0.208
This particle will travel -0.388 MFPs in the x-direction.
The distance to the boundary is 2.199 MFPs.
The particle collides at x = -2.587
Score! Particle with weight 0.5 added to the tally.
	Current score: 	 Region 0: 1.5   Region 1: 0.0
The particle is survival biased, now with weight 0.25
Rouletting the particles...
Phew. Particle survived, and now it has weight 1
The particle scatters, and is now traveling with mu = -0.197
This particle will travel -0.017 MFPs in the x-direction.
The distance to the boundary is 2.587 MFPs.
The particle collides at x = -2.604
Score! Particle with w

Random seed 4 gives a much more dynamic plot...

In [8]:
random.seed(4)
N = 1
tally = np.zeros(2)
p = particle(verbose=True)
p.transport()
tally += p.score

The particle was born at x = -1 cm (region 0), with weight 1.
This particle will travel -1.199 MFPs in the x-direction.
The distance to the boundary is 1.0 MFPs.
The particle collides at x = -2.199
Score! Particle with weight 1 added to the tally.
	Current score: 	 Region 0: 1.0   Region 1: 0.0
The particle is survival biased, now with weight 0.5
The particle scatters, and is now traveling with mu = -0.208
This particle will travel -0.388 MFPs in the x-direction.
The distance to the boundary is 2.199 MFPs.
The particle collides at x = -2.587
Score! Particle with weight 0.5 added to the tally.
	Current score: 	 Region 0: 1.5   Region 1: 0.0
The particle is survival biased, now with weight 0.25
Rouletting the particles...
Phew. Particle survived, and now it has weight 1
The particle scatters, and is now traveling with mu = -0.197
This particle will travel -0.017 MFPs in the x-direction.
The distance to the boundary is 2.587 MFPs.
The particle collides at x = -2.604
Score! Particle with w